<a href="https://colab.research.google.com/github/ShawnLiu119/CapstoneProject_BankingMarketing/blob/main/BankMkting_classificationmodelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd 
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,r2_score
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
import warnings
from imblearn.over_sampling import SMOTE
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/ShawnLiu119/CapstoneProject_BankingMarketing/main/rawdata/bank-additional-full.csv'
data = pd.read_csv(url, sep=';')

In [3]:
data['education'].value_counts()

data_1 = data.copy()
data_1 = data_1[data_1['education'] != 'illiterate']

data_1['edu_new'] = data_1['education'].map({'basic.4y': 'pre_high', 
                                         'basic.9y': 'pre_high',
                                         'basic.6y': 'pre_high',
                                         'university.degree' : 'post_high',
                                         'professional.course':'post_high',
                                         'high.school' : 'high',
                                         'unknown' :'unknown' })

In [4]:
def map_job(row):
    if row['job'] in ['admin.', 'management', 'entrepreneur','self-employed']:
        cat = 'white_collar'
    elif row['job'] in ['housemaid', 'blue-collar', 'technician', 'services']:
        cat = 'blue_collar'
    elif row['job'] in ['student', 'retired', 'unemployed']:
        cat = 'not_working'
    else:
        cat = 'unknown'
    return cat

data_1['job_new'] = data_1.apply(map_job, axis=1)
data_1['job_new'].value_counts(dropna = False)

blue_collar     21017
white_collar    16217
not_working      3606
unknown           330
Name: job_new, dtype: int64

In [5]:
def map_month(row):
    if row['month'] in ['mar', 'apr', 'may']:
        cat = 'spring'
    elif row['month'] in ['jun', 'jul', 'aug']:
        cat = 'summer'
    elif row['month'] in ['sep', 'oct', 'nov']:
        cat = 'fall'
    else:
        cat = 'winter'
    return cat

data_1['season'] = data_1.apply(map_month, axis=1)
data_1['season'].value_counts(dropna = False)

summer    18660
spring    16942
fall       5386
winter      182
Name: season, dtype: int64

In [6]:
data_1.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'edu_new', 'job_new',
       'season'],
      dtype='object')

### Data Transformation - V1 (Agreed on Team Discussion)

In [7]:
#This transformed dataframe is based on group discussion on 7/23
# - keep all numerical features as they are
# - consolidate categorical features (education, job, season) and drop originial ones

df_tran_1 = data_1[['age', 'marital', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'edu_new', 'job_new', 'season']]

In [23]:
df_tran_1.shape
#this is because we removed illiterate

(41170, 20)

In [8]:
df_tran_1 = df_tran_1.drop(['duration'], axis=1)
df_tran_1.columns

Index(['age', 'marital', 'default', 'housing', 'loan', 'contact',
       'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'edu_new', 'job_new', 'season'],
      dtype='object')

### One Hot Encoding For Categorical Features

In [20]:
y = df_tran_1['y'].apply(lambda x: 0 if x=='no' else (1 if x=='yes' else -1))
y.value_counts()

0    36534
1     4636
Name: y, dtype: int64

In [31]:
x = df_tran_1.drop('y', axis=1)

(41170, 19)

In [28]:
df_raw = pd.get_dummies(x)
df_raw.columns

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'edu_new_high',
       'edu_new_post_high', 'edu_new_pre_high', 'edu_new_unknown',
       'job_new_blue_collar', 'job_new_not_working', 'job_new_unknown',
       'job_new_white_collar', 'season_fall', 'season_spring', 'season_summer',
       'season_winter'],
      dtype='object')

In [30]:
#drop the unknow columns as they are correlated to other categories

df_raw = df_raw.drop(['marital_unknown', 'default_unknown', 'housing_unknown', 'loan_unknown', 'edu_new_unknown', 'job_new_unknown'], axis=1)
df_raw.shape

(41170, 38)

### Train-Test Split

In [33]:
x_train,x_test,y_train,y_test = train_test_split(df_raw, y, random_state=42)
print(x_train.shape,
      x_test.shape,
      y_train.shape,
      y_test.shape)

NameError: ignored

imbalanced dataset - fewer yes, more no

https://towardsdatascience.com/all-about-imbalanced-machine-learning-classifiers-60563014d2b3


In [14]:
pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 27.6 MB/s 
     |████████████████████████████████| 199 kB 65.7 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1


In [ ]:
df_

In [15]:
import imblearn

### Data Transformation - V2 (Shawn's attempt based on FAMDD analysis)
##### if you are focusing on the agreed v1, feel free to skip this part, just want to experiment, may not have too signficant impact

In [ ]:
data_2 = data.copy()

In [ ]:
#findings from FAMD analysis
# there is significant distinguishablity for feature - poutcome / contact / default, so no need to combine any features
# job -> separate student and retired stand alone as they are distinguished from others
# housing, loan are alomost overlapping for all segment, not meaningful to consolidate

In [ ]:
data_2['edu_new'] = data_2['education'].map({'basic.4y': 'pre_colleage', 
                                         'basic.9y': 'pre_colleage',
                                         'basic.6y': 'pre_colleage',
                                         'university.degree' : 'post_colleage',
                                         'professional.course':'post_colleage',
                                         'illiterate': 'illiterate',
                                         'high.school' : 'pre_colleage',
                                         'unknown' :'unknown' })

data_2['edu_new'].value_counts(dropna = False)

pre_colleage     22028
post_colleage    17411
unknown           1731
illiterate          18
Name: edu_new, dtype: int64

In [ ]:
def map_job_2(row):
    if row['job'] in ['admin.', 'management', 'entrepreneur','self-employed', 'housemaid', 'blue-collar', 'technician', 'services', 'unemployed']:
        cat = 'others'
    elif row['job'] in ['retired']:
        cat = 'retired'
    elif row['job'] in ['student']:
        cat = 'student'
    else:
        cat = 'unknown'
    return cat

data_2['job_new'] = data_2.apply(map_job_2, axis=1)
data_2['job_new'].value_counts(dropna = False)

others     38263
retired     1720
student      875
unknown      330
Name: job_new, dtype: int64

In [ ]:
def map_marri(row):
    if row['job'] in ['divorced', 'married', 'unknown']:
        cat = 'not_single'
    else:
        cat = 'single'
    return cat

data_2['marr_new'] = data_2.apply(map_marri, axis=1)
data_2['marr_new'].value_counts(dropna = False)

single        40858
not_single      330
Name: marr_new, dtype: int64

In [ ]:
def map_month_2(row):
    if row['month'] in ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov']:
        cat = 'non_winter'
    else:
        cat = 'winter'
    return cat

data_2['season_new'] = data_2.apply(map_month_2, axis=1)
data_2['season_new'].value_counts(dropna = False)

non_winter    41006
winter          182
Name: season_new, dtype: int64

In [ ]:
df_tran_2 = data_2.drop(['job', 'marital', 'education', 'month'], axis=1)